# Equilibrium and Present Value

#### Written for the QuantEcon Africa Workshop (July 2023)
#### Author: [John Stachurski](http://johnstachurski.net/) and Frank

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import newton

## Equilibrium
### The Market

We consider a market for coffee beans.  The price per kilo is $p$.  Total supply at price $p$ is

$$ q_s (p) = b \sqrt{p} $$

and total demand is 

$$ q_d (p) = a \frac{1}{p}, $$

where $a$ and $b$ are positive parameters.

Now let's write routines to compute supply and demand as functions of price and parameters:

In [ ]:
def supply(p, b):
    return b * np.sqrt(p)

def demand(p, a):
    return a * (1/p)

### Exercise 1

Plot both supply and demand as functions of $p$ when $a=1$ and $b=0.5$.  Plot for prices in the interval from $0.2$ up to $4$.  Although it is unusual, put price on the horizonal axis.  Use a legend to label the two functions and be sure to label the axes.  Looking at the picture, make a rough estimate of the equilibrium price,
where demand equals supply.

### Solution

In [ ]:
fig, ax = plt.subplots()
p_grid = np.linspace(0.2, 4, 200)
ax.plot(p_grid, supply(p_grid, 0.5), label='supply')
ax.plot(p_grid, demand(p_grid, 1.0), label='demand')
ax.set_xlabel("price")
ax.set_ylabel("quantity")
ax.legend()
plt.show()

The equilibrium price looks to be about 1.6.

### Exercise 2

Write a function that takes arguments $a, b, p$ and returns *excess demand*, which is defined as

$$ e(p) = q_d(p) - q_s(p) $$


### Solution

In [ ]:
def excess_demand(p, a, b):
    return demand(p, a) - supply(p, b)

### Exercise 3

Write a class definition that creates a class called `Market` with four methods:

1. An `__init__` method that specifies default parameter values $a=1$ and $b=0.5$.
2. A `demand` method that returns $q_d(p)$ given $p$
3. A `supply` method that returns $q_s(p)$ given $p$
4. An `excess_demand` method that returns $q_d(p) - q_s(p)$ given $p$. 

Using an instance of the class, plot excess demand over the interval from $0.2$ up to $4$.  Also plot a horizontal line at zero.  The equilibrium price is where excess demand crosses zero.

In [ ]:
class Market:
    
    def __init__(self, a=1.0, b=0.5):
        self.a, self.b = a, b
        
    def supply(self, p):
        return self.b * np.sqrt(p)
    
    def demand(self, p):
        return self.a * (1/p)
    
    def excess_demand(self, p):
        return self.demand(p) - self.supply(p)

In [ ]:
market = Market()

fig, ax = plt.subplots()
p_grid = np.linspace(0.2, 4, 200)
ax.plot(p_grid, market.excess_demand(p_grid), label='excess demand')
ax.plot(p_grid, np.zeros_like(p_grid), 'k--')
ax.set_xlabel("price")
ax.set_ylabel("quantity")
ax.legend()
plt.show()

Next we'll write a function that takes an instance of the `Market` class (i.e., an object `market` constructed via `market = Market()`) and returns a market clearing price via Newton's method.  If necessary, adjust your code above so that this routine works.

In [ ]:
def compute_equilibrium(market, price_init=2.0):
    p_star = newton(market.excess_demand, price_init)
    return p_star

In [ ]:
market = Market()
compute_equilibrium(market)

### Exercise 4

For $b$ in a grid of 200 values between 0.5 and 1.0, plot the equilibrium price for each $b$.

Does the curve that you plotted slope up or down?  Try to provide an explanation for what you see in terms of market equilibrium.

### Solution

In [ ]:
b_grid = np.linspace(0.5, 1.0, 200)
prices = np.empty_like(b_grid)
for i, b in enumerate(b_grid):
    market = Market(b=b)
    prices[i] = compute_equilibrium(market)

In [ ]:
fig, ax = plt.subplots()
ax.plot(b_grid, prices, label="equilibrium prices")
ax.set_xlabel("$b$")
ax.set_ylabel("price")
ax.legend()
plt.show()

The curve slopes down because an increase in $p$ pushes up supply at any given price.  (In other words, the supply curve shifts up.)  With greater supply, the price tends to fall.

## Present Value
### Exercise 5: Computing Present Value

Imagine you have the opportunity to receive $500 two years from now. If the annual discount rate is 5%, what is the present value of this future cash flow?

*Hint: Use the formula of [present value](https://en.wikipedia.org/wiki/Present_value).*

In [ ]:
def present_value(FV, r, n):
    PV = FV / (1 + r)**n
    return round(PV, 2);

In [ ]:
try:
    assert abs(present_value(500, 0.05, 2) - 453.51) < 1e-2
    print("Test passed!")
except AssertionError:
    print("Test failed.")
except Exception as e:
    print(f"An error occurred during testing: {e}")

### Exercise 6: Evaluating a Series of Future Cash Flows

Suppose you are evaluating an investment that promises to pay $1000 at the end of each year for the next five years. If your discount rate is still 5% per annum, what is the present value of these future cash flows?

*Hint: In this case, you will have to sum the present values of each future cash flow.*

In [ ]:
def present_value_series(annual_cash_flow, r, years):
    result = sum([annual_cash_flow / (1 + r)**n for n in years])
    return round(result, 2);

In [ ]:
try:
    assert abs(present_value_series(1000, 0.05, range(1, 6)) - 4329.48) < 1e-2
    print("Test passed!")
except AssertionError:
    print("Test failed.")
except Exception as e:
    print(f"An error occurred during testing: {e}")

### Exercise 7: Asset Pricing Using the Present Value Model

Consider a stock that is expected to pay dividends of $100$ at the end of each year for the next three years, after which point it is expected to be sold for $1500$. If the annual discount rate is 6%, calculate:

1. The present value of the dividends.
2. The present value of the terminal price of the stock.
3. The total present value of the stock (which should be equal to its current price).

*Hint: Use the same method as in the second exercise for the dividends, and the method from the first exercise for the terminal price.*

In [ ]:
def calculate_stock_value(dividend, terminal_price, r, years):
    dividend = 100
    terminal_price = 1500
    r = 0.06
    years = range(1, 4)  # from 1 to 3 inclusive
    PV_dividends = sum([dividend / (1 + r)**n for n in years])
    PV_terminal = terminal_price / (1 + r)**3
    PV_total = PV_dividends + PV_terminal
    return round(PV_dividends, 2), round(PV_terminal, 2), round(PV_total, 2)

In [ ]:
try:
    PV_dividends, PV_terminal, PV_total = calculate_stock_value(100, 1500, 0.06, range(1, 4))
    assert abs(PV_dividends - 267.3) < 1e-2
    assert abs(PV_terminal - 1259.43) < 1e-2
    assert abs(PV_total - 1526.73) < 1e-2
    print("Test passed!")
except AssertionError:
    print("Test failed.")
except Exception as e:
    print(f"An error occurred during testing: {e}")

### Exercise 8: Matrix Multiplication in Asset Pricing

Consider an asset that is expected to yield dividends of $100$ at the end of each year for the next five years, after which point it is predicted to be sold for $2000$. The annual discount rate is set to be 5%. Your task is to compute:

1. The present value of the dividends using matrix multiplication.
2. The present value of the final price of the asset.
3. The total present value of the asset (which should be equal to its current price).

*Hint: Use matrix multiplication to solve for the present value of dividends, and use the same method as in the previous exercise for the final price.*

In [ ]:
def calculate_asset_value(dividend, final_price, discount_rate, years):
    dividend = np.array([100]*5)
    final_price = 2000
    discount_rate = 0.05
    years = np.array(range(1, 6))  # from 1 to 5 inclusive
    
    # TODO: Create a discount factor matrix
    discount_matrix = 1 / (1 + discount_rate) ** years

    # TODO: Calculate the present value of dividends using matrix multiplication
    PV_dividends = np.dot(dividend, discount_matrix)

    # TODO: Calculate the present value of the final price
    PV_final_price = final_price * (1 / (1 + discount_rate) ** years[-1])

    # Calculate the total present value
    PV_total = PV_dividends + PV_final_price

    return round(PV_dividends, 2), round(PV_final_price, 2), round(PV_total, 2)

In [ ]:
try:
    PV_dividends, PV_final_price, PV_total = calculate_asset_value(100, 2000, 0.05, range(1, 6))
    assert abs(PV_dividends - 432.95) < 1e-2
    assert abs(PV_final_price - 1567.05) < 1e-2
    assert abs(PV_total - 2000.0) < 1e-2
    print("Test passed!")
except AssertionError:
    print("Test failed.")
except Exception as e:
    print(f"An error occurred during testing: {e}")

### Exercise 9

Read section [present value calculation](https://intro.quantecon.org/pv.html#present-value-calculations) before attempting exercises 8.1 and 8.2.

Let $A, p, d, b$ be as defined in that section.

So we have

$$
    A p = d + b
$$

#### Exercise 9.1 (Price of a worthless stock)

Compute and plot the corresponding asset price sequence when $T=6$, $p^*_{T+1} = 0$, $d_t=0$ and $\delta
= 0.98$.

#### Solution

In [ ]:
T = 6
p_star = 0.0
δ = 0.98

In [ ]:
d = np.zeros(T+1)
A = np.zeros((T+1, T+1))
for i in range(T+1):
    for j in range(T+1):
        if i == j:
            A[i, j] = 1
            if j < T:
                A[i, j+1] = -δ

b = np.zeros(T+1)
b[-1] = δ * p_star
p = np.linalg.solve(A, d + b)

In [ ]:
fig, ax = plt.subplots()
ax.plot(p, 'o-', ms=4, alpha=0.8, label='asset price')
ax.legend()
ax.set_xlabel('time')
plt.show()

#### Exercise 9.2 (Gorden growth formula)

Let $\delta$ be the same as Exercise 9.1.

Compute and plot the corresponding asset price sequence when $T=100$, $p^*_{T+1} = g^{T+1} d_0$ and $d_t=g^t d_0$
where $g=1.1$ and $d_0=0.1$.

#### Solution

In [ ]:
T = 100
δ = 0.98
g = 1.1
current_d = 0.1 # d_0

In [ ]:
p_star = g**(T+1) * current_d

d = []
for t in range(T+1):
    d.append(current_d)
    current_d = current_d * 1.1**t 

In [ ]:
A = np.zeros((T+1, T+1))
for i in range(T+1):
    for j in range(T+1):
        if i == j:
            A[i, j] = 1
            if j < T:
                A[i, j+1] = -δ

b = np.zeros(T+1)
b[-1] = δ * p_star
p = np.linalg.solve(A, d + b)

In [ ]:
fig, ax = plt.subplots()
ax.plot(p, 'o-', ms=4, alpha=0.8, label='asset price')
ax.legend()
ax.set_xlabel('time')
plt.show()

#### Exercise 9.3 (Analytical expressions)

Read section [analytical expressions](https://intro.quantecon.org/pv.html#analytical-expressions).

Give analytical expressions for the asset price $p_t$ in Exercise 9.2.

#### Solution

$$
   p_t =  \sum_{s=t}^T \delta^{s-t} g^s d_0 +  \delta^{T+1-t} g^{T+1} d_0
$$
